[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nina-Konovalova/WildHack_Tags/blob/main/specifying_tag_generation_attribute_clustering.ipynb)

##### Installation

In [1]:
! pip install --upgrade spacy
! python -m spacy download ru_core_news_lg

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


##### Download

In [4]:
!gdown --id 1Fxeo4ewOaqBNgOtGOE8lqnAxTlPsGi7A # scraped results for all products 'капри' and all attributes

!gdown --id 1xI8vAlZQWCCR-ne1J2gX3XopuqztmRaK # scraped results for all products 'капри' and entities
                                                                                        # 'Цвет' 
                                                                                        # 'Страна производителя' 
                                                                                        # 'Назначение' 
                                                                                        # 'Тип кармана'

Downloading...
From: https://drive.google.com/uc?id=1Fxeo4ewOaqBNgOtGOE8lqnAxTlPsGi7A
To: /content/kapri_product.json
100% 1.40M/1.40M [00:00<00:00, 21.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xI8vAlZQWCCR-ne1J2gX3XopuqztmRaK
To: /content/kapri_entities.json
100% 751k/751k [00:00<00:00, 11.7MB/s]


##### Imports

In [21]:
import json
from collections import Counter
import numpy as np
import pandas as pd
import string

from sklearn.cluster import KMeans, AgglomerativeClustering
import spacy

##### Scan of  attributes for product with title 'капри'

In [7]:
with open('/content/kapri_product.json') as f:
    data = json.load(f)

In [11]:
attributes = [item['attribute'] for item in data]
attributes = [a for item in attributes for a in item]
attributes_cnt = Counter(attributes)
attributes = [{"attribute": item, "cnt": attributes_cnt[item]} for item in set(attributes)]

In [12]:
attributes

[{'attribute': 'Декоративные элементы', 'cnt': 737},
 {'attribute': 'Уход за вещами', 'cnt': 1232},
 {'attribute': 'Любимые герои', 'cnt': 27},
 {'attribute': 'Спортивное назначение', 'cnt': 17},
 {'attribute': 'Утеплитель', 'cnt': 25},
 {'attribute': 'Материал подкладки', 'cnt': 8},
 {'attribute': 'Рисунок', 'cnt': 381},
 {'attribute': 'Особенности модели', 'cnt': 783},
 {'attribute': 'Размер на модели', 'cnt': 591},
 {'attribute': 'Назначение', 'cnt': 1321},
 {'attribute': 'Крой', 'cnt': 1058},
 {'attribute': 'Страна производства', 'cnt': 1693},
 {'attribute': 'Параметры модели на фото (ОГ-ОТ-ОБ)', 'cnt': 171},
 {'attribute': 'Возрастная группа (лет)', 'cnt': 178},
 {'attribute': 'Тип посадки', 'cnt': 1306},
 {'attribute': 'Длина по внутреннему шву', 'cnt': 605},
 {'attribute': 'Ограничение', 'cnt': 1},
 {'attribute': 'Комплектация', 'cnt': 1774},
 {'attribute': 'Фактура материала', 'cnt': 1072},
 {'attribute': 'Тип карманов', 'cnt': 1202},
 {'attribute': 'Вид застежки', 'cnt': 1278}

Let's consider those entities

In [ ]:
named_entities_ru = ['Назначение', 'Цвет', 'Страна производства', 'Тип карманов']
named_entities_eng = ['purpose', 'color', 'country', 'pocket_type']

##### Cluster entities separately

In [22]:
nlp = spacy.load('ru_core_news_lg')

In [15]:
with open('/content/kapri_entities.json') as f:
    db = json.load(f)

Страна производителя

In [19]:
country = [item['country'] for item in db if item['country']]
country = list(set(country))
country

['Таиланд',
 'Латвия',
 'Индия',
 'Бангладеш',
 'Италия',
 'Германия',
 'Киргизия',
 'Словакия',
 'Россия',
 'Пакистан',
 'Греция',
 'Индонезия',
 'Египет',
 'Китай',
 'Албания',
 'Казахстан',
 'Испания',
 'Вьетнам',
 'Турция',
 'Узбекистан',
 'Франция',
 'Сербия',
 'Украина',
 'Соединенные Штаты',
 'Камбоджа',
 'Румыния',
 'Республика Македония',
 'Тунис',
 'Польша',
 'Финляндия',
 'Бразилия',
 'Беларусь',
 'Гонконг']

Цвет

In [20]:
color = [item['color'] for item in db if item['color']]
color = [item.split(', ') for item in color if item]
color = [i for item in color for i in item if i]
color = list(set(color))
color

['бежевый',
 'красный',
 'серо-голубой',
 'коричневый',
 'розовая гвоздика',
 'ярко-розовый',
 'светло-коричневый',
 'коричнево-серый',
 'розовато-серый',
 'оранжевый',
 'кирпично-красный',
 'серо-синий',
 'светло-голубой',
 'тускло-сиреневый',
 'темно-оливковый',
 'сливовый',
 'маренго',
 'охра',
 'лазурный',
 'аметистовый',
 'серый',
 'темно-красный',
 'сумеречный',
 'темно-серый меланж',
 'алый',
 'корица',
 'электрик',
 'антрацитовый',
 'черничный',
 'джинс',
 'светло-фиолетовый',
 'ванильный',
 'розовый',
 'коралловый',
 'черный меланж',
 'персиковый',
 'желтый',
 'лиловый',
 'темно-бирюзовый',
 'розово-коричневый',
 'огненно-красный',
 'темно-голубой',
 'темный хаки',
 'темная зеленая оливка',
 'бирюзовый',
 'темно-синий',
 'мятный',
 'глубокий коралловый',
 'синяя лазурь',
 'светло-синий',
 'иссиня-черный',
 'кремовый',
 'травяной зеленый',
 'ярко-красный',
 'темно-фиолетовый',
 'малиновый',
 'салатовый',
 'глубокий коричневый',
 'лавандовый',
 'оливковый',
 'серо-бежевый',
 'го

In [30]:
X = np.array([nlp(word).vector / nlp(word).vector_norm for word in color if nlp(word).vector_norm])

color_model = AgglomerativeClustering(n_clusters=50, linkage="average", affinity='cosine').fit(X)

print('Labels')
print(color_model.labels_)

cluster_id = 23
print('Cluster', cluster_id)
np.array([word for word in color if nlp(word).vector_norm])[color_model.labels_ == cluster_id]

Labels
[ 4  0  5  4  6  5  5  5  5  0  0  5  5  5  5  2 39 32 21 31  5  5 37 23
  0  6 25 24 47  5 41  5  7 23  2  0  0  5  4  0  5 14  9 21  5 46  7  9
  5  0  4  0  5  5  2 19  4  1 42  5 40 19  5  0  5  4 26  5 12  5  5 10
 43 15  5  1  2  5  5 35  4 23  2  5  5  5  0  5 33 34  9  5 44 27  5  0
  0 40  8  3  5 17  0  0  5  5 45 11 30 28  5  0  5 17  5  1  0  5 38  5
 20 16  5  5 29 49 22  5 36 18  3  0 11  5 24  5  3 14 48  0  5 13  5  8
  0 23 21  0  5  0  0 10  5]
Cluster 23


array(['темно-серый меланж', 'черный меланж', 'сиреневый меланж',
       'серый меланж'], dtype='<U22')

In [29]:
purpose = [item['purpose'] for item in db if item['purpose']]
purpose = [''.join(',' if c in string.punctuation else c for c in item) for item in purpose if item]
purpose = [item.split(', ') for item in purpose if item]
purpose = [i for item in purpose for i in item if i]
purpose = [item.split(',') for item in purpose if item]
purpose = [i for item in purpose for i in item if i]
purpose = list(set(purpose))

In [32]:
X = np.array([nlp(word).vector / nlp(word).vector_norm for word in purpose if nlp(word).vector_norm])

purpose_model = AgglomerativeClustering(n_clusters=50, linkage="average", affinity='cosine').fit(X)

print('Labels')
print(purpose_model.labels_)

cluster_id = 7
print('Cluster', cluster_id)
np.array([word for word in purpose if nlp(word).vector_norm])[purpose_model.labels_ == cluster_id]

Labels
[ 5  4  1 18 22  5 16 48 12  7  0  2 43 45  7  9 39  6  4  5 47  5  5 11
  5 10  1  5  3 12 44 34 18  1  1 31  5  0  7  2  8  8  4 22 22 32 12  4
 49  5  0  8 28  8 34  3  9 23  4 20 21 35  7 20 19 24 10 18 15  5  2 26
  5  7  7 25  2  3  2  1  0 33 26  5 42 27 14 41  5  5  2  5  5  5  8  2
  4  5  4  5  5 16 30  4 29 18  9 18 36 14 15  9  4  3  1 11 25 25  4 17
 38 18  7 18  9  5 19 37 13  5 34  5 46  6  4  9  5  2 40 13]
Cluster 7


array(['отдых', 'отдых и путешествия', 'Спорт и отдых',
       'отдых и прогулка', 'активный отдых', 'на отдых',
       'прогулки и отдых'], dtype='<U45')

In [33]:
pocket_type = [item['pocket_type'] for item in db if item['pocket_type']]
pocket_type = [item.split('; ') for item in pocket_type  if item]
pocket_type = [i for item in pocket_type  for i in item if i]
pocket_type = list(set(pocket_type))

In [37]:
X = np.array([nlp(word).vector / nlp(word).vector_norm for word in pocket_type if nlp(word).vector_norm])

pocket_type_model = AgglomerativeClustering(n_clusters=10, linkage="average", affinity='cosine').fit(X)

print('Labels')
print(pocket_type_model.labels_)

cluster_id = 2
print('Cluster', cluster_id)
np.array([word for word in pocket_type if nlp(word).vector_norm])[pocket_type_model.labels_ == cluster_id]

Labels
[1 9 1 2 1 2 0 1 2 1 0 1 1 5 5 1 0 1 7 4 2 1 0 1 5 8 3 5 1 0 2 1 2 1 6 1 1
 1 0]
Cluster 2


array(['в шве', 'В боковом шве штанов', 'накладной в шве',
       'в боковах швах', 'в боковом шве', 'в шве, накладные сзади'],
      dtype='<U32')

##### Generate top-k tags for random query from list

In [42]:
clarifying_tags = []
for a in country:
    clarifying_tags.append('капри ' + a)

vocab = np.array([word for word in color if nlp(word).vector_norm])
for i in range(color_model.n_clusters_):
    t = np.random.choice(vocab[color_model.labels_ == i])
    clarifying_tags.append('капри ' + t.lower())

vocab = np.array([word for word in purpose if nlp(word).vector_norm])
for i in range(purpose_model.n_clusters_):
    t = np.random.choice(vocab[purpose_model.labels_ == i])
    clarifying_tags.append('капри ' + t.lower())

vocab = np.array([word for word in pocket_type if nlp(word).vector_norm])
for i in range(pocket_type_model.n_clusters_):
    t = np.random.choice(vocab[pocket_type_model.labels_ == i])
    clarifying_tags.append('капри ' + t.lower())

In [43]:
clarifying_tags

['капри Таиланд',
 'капри Латвия',
 'капри Индия',
 'капри Бангладеш',
 'капри Италия',
 'капри Германия',
 'капри Киргизия',
 'капри Словакия',
 'капри Россия',
 'капри Пакистан',
 'капри Греция',
 'капри Индонезия',
 'капри Египет',
 'капри Китай',
 'капри Албания',
 'капри Казахстан',
 'капри Испания',
 'капри Вьетнам',
 'капри Турция',
 'капри Узбекистан',
 'капри Франция',
 'капри Сербия',
 'капри Украина',
 'капри Соединенные Штаты',
 'капри Камбоджа',
 'капри Румыния',
 'капри Республика Македония',
 'капри Тунис',
 'капри Польша',
 'капри Финляндия',
 'капри Бразилия',
 'капри Беларусь',
 'капри Гонконг',
 'капри черный матовый',
 'капри лавандовый',
 'капри морковный',
 'капри кофе с молоком',
 'капри кремовый',
 'капри темно-коричневый',
 'капри розовая гвоздика',
 'капри коралловый',
 'капри зеленое море',
 'капри зеленая оливка',
 'капри шоколадный',
 'капри фисташковый',
 'капри кукурузный',
 'капри полынь',
 'капри темный хаки',
 'капри cиний лён',
 'капри песочный',
 'ка

In [55]:
query = 'капри и блузка летние'
def choose_top_k(tags, k=7):
    # the most similar
    return np.random.choice(tags, k)

'Query', query, 'Clarifying_tags', *choose_top_k(clarifying_tags)

('Query',
 'капри и блузка летние',
 'Clarifying_tags',
 'капри Соединенные Штаты',
 'капри Индия',
 'капри бег',
 'капри большие размеры ',
 'капри фуксия',
 'капри Албания',
 'капри карманы сбоку')